In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix


In [ ]:
# Fetch SpaceX launch data from the SpaceX API
url = "https://api.spacexdata.com/v4/launches"
response = requests.get(url)
data = response.json()

# Normalize data into a DataFrame
df = pd.json_normalize(data)
df.head()


In [ ]:
# Select relevant features
columns_of_interest = [
    'name', 'date_utc', 'success', 'rocket', 'payloads', 'launchpad', 
    'cores', 'crew', 'flight_number'
]
df = df[columns_of_interest]

# Drop rows with missing success values
df = df[df['success'].notna()]

# Convert launch date
df['date'] = pd.to_datetime(df['date_utc'])
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month

# Encode success as binary
df['success'] = df['success'].astype(int)

# Simplify by using only the year, month, and flight number for now
df_model = df[['year', 'month', 'flight_number', 'success']]
df_model.head()


In [ ]:
# EDA: Visualizing correlations
sns.heatmap(df_model.corr(), annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()


In [ ]:
# Split into features and target
X = df_model.drop('success', axis=1)
y = df_model['success']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Random Forest Classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Predictions
y_pred = clf.predict(X_test)

# Evaluation
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
